In [5]:
# 安装pdfminer
!pip install pdfminer.six

In [22]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine, LTChar, LTImage


# 读取 PDF 文件
file_directory = "not_upload\\关于开展公众市场三季度营销活动的通知.pdf"

print(os.getcwd())
# 获取当前目录
current_directory = os.getcwd()
# 获取当前目录的上上层目录
grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))

file_path = os.path.join(grandparent_directory, file_directory)
print(file_path)




# 定义一个函数来提取并排序文本
def extract_and_sort_text(file_path):
    all_text = []

    # 提取每一页的布局信息
    for page_layout in extract_pages(file_path):
        # 按位置排序布局元素
        print(page_layout)
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)
        print(sorted_elements)

        # 合并文本
        current_text = ""
        for element in sorted_elements:
            if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                current_text += element.get_text().replace(" ", "")
            elif isinstance(element, LTImage):
                print("Image at position:", element.bbox)

        all_text.append(current_text.strip())

    return all_text


#提取并排序文本
all_text = extract_and_sort_text(file_path)

# 打印每一页的文本
for i, text in enumerate(all_text):
    print(f"Page {i + 1}:")
    print(text)
    print("\n")


The PDF <_io.BufferedReader name='c:\\Users\\Administrator\\Documents\\source\\Learn-LLM\\not_upload\\关于开展公众市场三季度营销活动的通知.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


c:\Users\Administrator\Documents\source\Learn-LLM\99-project\RAG
c:\Users\Administrator\Documents\source\Learn-LLM\not_upload\关于开展公众市场三季度营销活动的通知.pdf
<LTPage(1) 0.000,0.000,595.300,841.900 rotate=0>
[<LTTextBoxHorizontal(0) -2379.759,-1360.026,2351.543,3372.247 'hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    \nhb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    \nhb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36    hb-qinyf 2024-08-13 11:09:36  

### pdf页面读取时，保证读取顺序
```python
def extract_and_sort_text(file_path):
    all_text = []

    # 提取每一页的布局信息
    for page_layout in extract_pages(file_path):
        # 按位置排序布局元素
        print(page_layout)
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)

        # 合并文本
        current_text = ""
        for element in sorted_elements:
            if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                current_text += element.get_text()
            elif isinstance(element, LTImage):
                print("Image at position:", element.bbox)

        all_text.append(current_text.strip())

    return all_text
```

`sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)` 

这段代码用来保证读取的pdf文档内容按顺序呈现，通过这种方式排列布局元素的原因有以下几点：
1. 在 pdfminer 中，元素坐标系统如下：
    - 原点 (0, 0)：位于页面的左下角。
    - x 轴：从左向右增加，一直到X轴边界。
    - y 轴：从下向上增加，一直到Y轴边界。
2. 排序目标
    - 首先按照 y1大小顺序， 将元素在Y轴上从上到下排序。reverse=True，即在Y轴方向上从PDF的上边界往Y轴的原点方向排列。
    - 当不同元素的y1相同时，则按x0从小到大顺序排列，即在X轴方向从PDF的X轴初始方向往右边界排列。
    - 经过以上在X，Y轴方向的排序步骤之后，sorted_elements 列表中存储的元素顺序符合人类查看PDF的习惯，从上到下，从左到右。


In [35]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextLine, LTTextBox,LAParams
from collections import Counter

# 读取 PDF 文件
file_directory = "not_upload\\关于开展公众市场三季度营销活动的通知.pdf"

# 获取当前目录
current_directory = os.getcwd()
print(f"当前目录: {current_directory}")

# 获取当前目录的上上层目录
grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))
print(f"上上层目录: {grandparent_directory}")

# 拼接文件路径
file_path = os.path.join(grandparent_directory, file_directory)
print(f"文件路径: {file_path}")

def identify_first_line_indent(page_layout):
    """
    识别页面中的首行缩进。
    """
    # 筛选文本行
    lines = [obj for obj in page_layout if isinstance(obj, LTTextBox) or isinstance(obj, LTTextLine)]
    print(f"文本行数量: {len(lines)}")
    
    # 按 y 坐标排序文本行
    lines.sort(key=lambda line: line.bbox[1], reverse=True)
    
    indent_results = []
    
    # 遍历每一行文本
    for i, line in enumerate(lines):
        # if i == 0:
        #     # 第一行不计算缩进
        #     continue
        
        prev_line = lines[i-1]
        
        # 计算当前行和前一行的 x 起始位置
        current_x_start = line.bbox[0]
        # prev_x_start = prev_line.bbox[0]
        # # 每页面第一行 x 起始位置
        # prev_x_start = lines[0].bbox[0]
        # 每页面 x轴的起始位置0
        prev_x_start = 0
        
        # 计算首行缩进
        indent = current_x_start - prev_x_start
        
        indent_results.append((line.get_text().strip(), indent))
    
    return indent_results


# 统计页面元素中文本行各种缩进值的个数，取个数最多且缩进值最大缩进值，且不为0
def filter_max_indent(indents):
    indent_counts = Counter(indent for _, indent in indents)
    # for i, count in indent_counts.items():
    #     print(f"首行缩进: {i:.2f} 像素, 出现次数: {count}")
    # 将统计出的结果dict，按出现的次数降序排序，即按value排序，item[1]是value
    sorted_itmes_by_value = sorted(indent_counts.items(), key = lambda item:item[1], reverse=True)

    # 取出现频次最高的前三组键值对（已转成tuple）
    max3_items_by_value = sorted_itmes_by_value[:3]

    # 对出现频次最高的前三对键值对（已转成tuple），再按第一个元素降序排序，item[0]是第一个元素
    sorted_itmes_by_key = sorted(max3_items_by_value,key = lambda item:item[0],reverse=True)

    # 取第一个元素最大的tuple对的第一个元素
    indent = sorted_itmes_by_key[0]
    print(f'当前页的缩进值是：{indent}')

# 读取 PDF 页面
for page_layout in extract_pages(file_path, laparams=LAParams()):
    print(f"页面布局对象: {page_layout}")
    

    # indents = identify_first_line_indent(page_layout)
    sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)
    indents = identify_first_line_indent(sorted_elements)
    print(f"首行缩进结果数量: {len(indents)}")
    
    if len(indents) == 0:
        print("没有检测到首行缩进。")
    else:
        for index, (text, indent) in enumerate(indents):
            # if indents[index][1] - indents[index-1][1] > 0:
            #     text = f"\t{text}" 
            filter_max_indent(indents) 
            # indent_counts = Counter(indent for _, indent in indents)
            # for i, count in indent_counts.items():
            #     print(f"首行缩进: {i:.2f} 像素, 出现次数: {count}")
                
            print(f"文本: '{text}', 首行缩进: {indent:.2f} 像素")
    
    # 打印每行文本的详细信息
    lines = [obj for obj in page_layout if isinstance(obj, LTTextLine)]
    for line in lines:
        print(f"文本: '{line.get_text().strip()}', bbox: {line.bbox}")

当前目录: c:\Users\Administrator\Documents\source\Learn-LLM\99-project\RAG
上上层目录: c:\Users\Administrator\Documents\source\Learn-LLM
文件路径: c:\Users\Administrator\Documents\source\Learn-LLM\not_upload\关于开展公众市场三季度营销活动的通知.pdf


The PDF <_io.BufferedReader name='c:\\Users\\Administrator\\Documents\\source\\Learn-LLM\\not_upload\\关于开展公众市场三季度营销活动的通知.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


页面布局对象: <LTPage(1) 0.000,0.000,595.300,841.900 rotate=0>
文本行数量: 17
首行缩进结果数量: 17
当前页的缩进值是：(190.44, 1)
文本: '联通宜昌营销字〔2024〕31 号', 首行缩进: 190.44 像素
当前页的缩进值是：(190.44, 1)
文本: '关于开展公众市场三季度营销活动的通知', 首行缩进: 104.40 像素
当前页的缩进值是：(190.44, 1)
文本: '各区县分公司：', 首行缩进: 79.44 像素
当前页的缩进值是：(190.44, 1)
文本: '三季度是全年承上启下的关键节点，锁定三季度主营收入增', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44, 1)
文本: '长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织', 首行缩进: 79.44 像素
当前页的缩进值是：(190.44, 1)
文本: '开展公众市场三季度营销活动，具体内容如下：', 首行缩进: 79.44 像素
当前页的缩进值是：(190.44, 1)
文本: '一、活动时间', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44, 1)
文本: '2024 年 7 月 1 日-2024 年 9 月 30 日', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44, 1)
文本: '二、营销目标', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44, 1)
文本: '全市移网发展 7 万户，移网净增 9534 户；宽带发展 1.86 万', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44, 1)
文本: '户，宽带净增 10625 户；129 元及以上实收高套发展 7360 户，FTTR', 首行缩进: 79.44 像素
当前页的缩进值是：(190.44, 1)
文本: '发展 4600 户，奋力实现三季度收入预算完成率 75%。', 首行缩进: 79.44 像素
当前页的缩进值是：(190.44, 1)
文本: '三、营销部署', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44, 1)
文本: '（一）市场策略', 首行缩进: 109.44 像素
当前页的缩进值是：(190.44,

使用collections库统计出页面上所有文本行首行缩进的像素值，即各个像素值出现的次数。然后取出现次数最多的像素值（前三），再然后取缩进像素值最大的元素，以还原经过处理后的文本原有的段落结构

In [24]:
from collections import Counter

abc = [{'a': 3, 'b': 8, 'c': 1, 'd': 12,'e':12,'f':12,'g':1,'h':12,'i':3,'j':99,'k':99,'l':99}]

for k,v in abc[0].items():
    print(k,v)

# 统计字典中各种键的值出现的次数，这里统计的是value即v出现的次数    
cde = Counter(v for k,v in abc[0].items())
print(f'统计字典中值出现的次数 {cde}')

# 将统计出的结果dict，按出现的次数降序排序，即按value排序，item[1]是value
sorted_itmes_by_value = sorted(cde.items(), key = lambda item:item[1], reverse=True)

# 取出现频次最高的前三组键值对（已转成tuple）
max3_items_by_value = sorted_itmes_by_value[:3]

# 对出现频次最高的前三对键值对（已转成tuple），再按第一个元素降序排序，item[0]是第一个元素
sorted_itmes_by_key = sorted(max3_items_by_value,key = lambda item:item[0],reverse=True)

# 取第一个元素最大的tuple对的第一个元素
indent = sorted_itmes_by_key[0]
print(f'按统计结果字典键值对的value降序排序结果，{sorted_itmes_by_value}')
print(f'按第一次排序结果列表中的第一个元素再次降序排序，{sorted_itmes_by_key}')
print(f'取列表中元组第一个元素最大的元组的第一个元素{indent[0]}')

a 3
b 8
c 1
d 12
e 12
f 12
g 1
h 12
i 3
j 99
k 99
l 99
统计字典中值出现的次数 Counter({12: 4, 99: 3, 3: 2, 1: 2, 8: 1})
按统计结果字典键值对的value降序排序结果，[(12, 4), (99, 3), (3, 2), (1, 2), (8, 1)]
按第一次排序结果列表中的第一个元素再次降序排序，[(99, 3), (12, 4), (3, 2)]
取列表中元组第一个元素最大的元组的第一个元素99
